In [3]:
import pandas as pd
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
df = pd.read_csv('Camera3__22591589__20201201_121328986_1DLC_resnet50_RightingAug10shuffle1_750000.csv',header=None)
df = df.iloc[1:]
df.columns = df.iloc[0] #shift col

# get col name
colname = []
a = df.iloc[0].to_list()
b = df.iloc[1].to_list()

for i in range(len(a)):
    colname.append(a[i]+'_'+b[i])
    
df.columns = colname
df = df.iloc[2:]
df = df.reset_index(drop=True)

dforiginal = df

df

,bodyparts_coords,Left_whisker_x,Left_whisker_y,Left_whisker_likelihood,Right_whisker_x,Right_whisker_y,Right_whisker_likelihood,Back_nose_x,Back_nose_y,Back_nose_likelihood,...,Front_nose_likelihood,Front_neck_x,Front_neck_y,Front_neck_likelihood,Front_back_x,Front_back_y,Front_back_likelihood,Front_tail_base_x,Front_tail_base_y,Front_tail_base_likelihood
0,0,531.8839111328125,543.1377563476562,0.07366800308227539,641.473388671875,391.0619201660156,0.003921568393707275,618.8458251953125,186.75323486328125,0.30092698335647583,...,0.3465791642665863,522.4891967773438,235.74945068359375,0.3120996952056885,390.50555419921875,464.483154296875,0.30274736881256104,230.30157470703125,647.01708984375,0.3249618709087372
1,1,517.9859008789062,558.7775268554688,0.06458362936973572,1.0679593086242676,-1.4618334770202637,0.007561296224594116,619.82470703125,187.0495147705078,0.4991215169429779,...,0.27923864126205444,520.66455078125,250.48193359375,0.25600123405456543,404.4996643066406,525.734619140625,0.14077118039131165,231.44680786132812,645.7059936523438,0.3517100214958191
2,2,638.8343505859375,394.9728698730469,0.0348874032497406,0.3059558868408203,-2.289017677307129,0.0046742260456085205,620.370849609375,186.34552001953125,0.6159675717353821,...,0.27919232845306396,327.5443420410156,555.88623046875,0.09210380911827087,402.97833251953125,527.2236328125,0.11871811747550964,225.38424682617188,647.751220703125,0.48577776551246643
3,3,640.8779296875,390.4175109863281,0.04667961597442627,0.9430496692657471,-1.9733867645263672,0.006555497646331787,620.7879028320312,186.21141052246094,0.5105687379837036,...,0.20986595749855042,327.5500183105469,555.8984985351562,0.16743481159210205,276.85601806640625,499.0296325683594,0.10893529653549194,225.11102294921875,647.2763671875,0.518294632434845
4,4,2.951847791671753,0.46306657791137695,0.02305665612220764,0.6109409332275391,-1.5954351425170898,0.015362828969955444,620.4580688476562,186.30224609375,0.4477770924568176,...,0.23741942644119263,524.0358276367188,251.1944122314453,0.08484464883804321,396.8866271972656,472.11199951171875,0.06701716780662537,225.68092346191406,647.1483764648438,0.39712148904800415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8996,8996,609.1804809570312,293.5596618652344,0.006150871515274048,462.92041015625,313.10595703125,0.0028872787952423096,525.316650390625,214.71324157714844,0.8171646595001221,...,0.23027709126472473,493.577392578125,176.4312744140625,0.14492404460906982,262.4599609375,448.3201599121094,0.3925940990447998,213.729248046875,597.7305908203125,0.07579970359802246
8997,8997,600.8598022460938,305.3955078125,0.007705986499786377,463.59259033203125,306.63818359375,0.0040496885776519775,524.777099609375,214.55377197265625,0.860185980796814,...,0.28394749760627747,495.625244140625,174.4217987060547,0.2170400321483612,268.33197021484375,444.0977783203125,0.7036703824996948,213.6555633544922,596.94189453125,0.03289410471916199
8998,8998,600.6270141601562,305.07098388671875,0.008622318506240845,462.0447692871094,306.94110107421875,0.004223734140396118,525.0558471679688,214.6747283935547,0.8631124496459961,...,0.17945757508277893,495.7090759277344,175.00352478027344,0.18311700224876404,268.4820556640625,444.14459228515625,0.57407546043396,214.1175994873047,598.1265258789062,0.08272901177406311
8999,8999,601.3507080078125,306.4750061035156,0.007821798324584961,462.8655090332031,305.6276550292969,0.007901549339294434,525.376953125,214.91415405273438,0.8895518183708191,...,0.17476385831832886,495.1595764160156,175.81695556640625,0.16586801409721375,267.97369384765625,445.8540344238281,0.8582571744918823,215.07357788085938,596.7421264648438,0.08207106590270996


In [5]:
coltodrop = []
for i in colname:
    if not (i.endswith('_x') or i.endswith('_y')):
        coltodrop.append(i)
        
df = df.drop(coltodrop,axis=1)
dfkalman_col = df.columns
df

,Left_whisker_x,Left_whisker_y,Right_whisker_x,Right_whisker_y,Back_nose_x,Back_nose_y,Left_ear_x,Left_ear_y,Right_ear_x,Right_ear_y,...,End_tail_x,End_tail_y,Front_nose_x,Front_nose_y,Front_neck_x,Front_neck_y,Front_back_x,Front_back_y,Front_tail_base_x,Front_tail_base_y
0,531.8839111328125,543.1377563476562,641.473388671875,391.0619201660156,618.8458251953125,186.75323486328125,515.3037719726562,300.67431640625,672.8833618164062,321.1720886230469,...,780.2487182617188,417.21612548828125,42.98600387573242,803.8118286132812,522.4891967773438,235.74945068359375,390.50555419921875,464.483154296875,230.30157470703125,647.01708984375
1,517.9859008789062,558.7775268554688,1.0679593086242676,-1.4618334770202637,619.82470703125,187.0495147705078,515.0113525390625,302.0635986328125,672.8079833984375,319.5641174316406,...,779.6026000976562,417.32830810546875,41.44740295410156,803.73828125,520.66455078125,250.48193359375,404.4996643066406,525.734619140625,231.44680786132812,645.7059936523438
2,638.8343505859375,394.9728698730469,0.3059558868408203,-2.289017677307129,620.370849609375,186.34552001953125,672.1485595703125,317.49041748046875,672.3745727539062,319.45184326171875,...,779.413330078125,417.1746826171875,43.750301361083984,803.3983764648438,327.5443420410156,555.88623046875,402.97833251953125,527.2236328125,225.38424682617188,647.751220703125
3,640.8779296875,390.4175109863281,0.9430496692657471,-1.9733867645263672,620.7879028320312,186.21141052246094,672.0919189453125,317.45574951171875,672.3777465820312,319.8484802246094,...,779.641845703125,417.37060546875,42.31147384643555,803.8068237304688,327.5500183105469,555.8984985351562,276.85601806640625,499.0296325683594,225.11102294921875,647.2763671875
4,2.951847791671753,0.46306657791137695,0.6109409332275391,-1.5954351425170898,620.4580688476562,186.30224609375,672.119384765625,318.331787109375,672.3798828125,319.51385498046875,...,779.588134765625,417.8123779296875,43.47657012939453,803.447265625,524.0358276367188,251.1944122314453,396.8866271972656,472.11199951171875,225.68092346191406,647.1483764648438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8996,609.1804809570312,293.5596618652344,462.92041015625,313.10595703125,525.316650390625,214.71324157714844,569.3412475585938,180.46913146972656,163.13958740234375,441.5647888183594,...,779.545166015625,416.71563720703125,393.6142578125,189.0755157470703,493.577392578125,176.4312744140625,262.4599609375,448.3201599121094,213.729248046875,597.7305908203125
8997,600.8598022460938,305.3955078125,463.59259033203125,306.63818359375,524.777099609375,214.55377197265625,525.24951171875,211.99623107910156,163.71731567382812,440.90191650390625,...,779.098876953125,416.72998046875,479.0615539550781,105.35392761230469,495.625244140625,174.4217987060547,268.33197021484375,444.0977783203125,213.6555633544922,596.94189453125
8998,600.6270141601562,305.07098388671875,462.0447692871094,306.94110107421875,525.0558471679688,214.6747283935547,567.13720703125,180.32220458984375,163.70835876464844,441.2519226074219,...,779.2969360351562,416.45123291015625,479.05950927734375,105.33193969726562,495.7090759277344,175.00352478027344,268.4820556640625,444.14459228515625,214.1175994873047,598.1265258789062
8999,601.3507080078125,306.4750061035156,462.8655090332031,305.6276550292969,525.376953125,214.91415405273438,568.021484375,180.30511474609375,164.09410095214844,441.06353759765625,...,779.342529296875,416.1557922363281,395.3743591308594,188.01133728027344,495.1595764160156,175.81695556640625,267.97369384765625,445.8540344238281,215.07357788085938,596.7421264648438


In [6]:
c= set(a)
c.remove('bodyparts')
print(c)

{'Right_lateral', 'Back_tail_base', 'Left_whisker', 'Left_front_paw', 'End_tail', 'Right_front_paw', 'Front_back', 'Left_ear', 'Middle_tail', 'Right_hind_paw', 'Front_nose', 'Back_belly', 'Left_hind_paw', 'Front_tail_base', 'Left_lateral', 'Back_throat', 'Front_neck', 'Right_whisker', 'Right_ear', 'Back_nose'}


In [7]:
masterlist = {}

for i in c:
    masterlist[i] = list(zip(df[i+'_x'].astype('float32'), df[i+'_y'].astype('float32')))
    


In [8]:
def kalmanfunction(arr):
    measurements = np.array(arr)
    initial_state_mean = [measurements[0, 0],
                      0,
                      measurements[0, 1],
                      0]

    transition_matrix = [[1, 1, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 1],
                         [0, 0, 0, 1]]

    observation_matrix = [[1, 0, 0, 0],
                          [0, 0, 1, 0]]
    kf1 = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrix,
                  initial_state_mean = initial_state_mean)
    
    kf1 = kf1.em(measurements, n_iter=5)
    
    kf2 = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrix,
                  initial_state_mean = initial_state_mean,
                  observation_covariance = 10*kf1.observation_covariance,
                  em_vars=['transition_covariance', 'initial_state_covariance'])
    
    kf2 = kf2.em(measurements, n_iter=5)
    (smoothed_state_means, smoothed_state_covariances)  = kf2.smooth(measurements)

    return list(zip(smoothed_state_means[:,0],smoothed_state_means[:,2]))

In [9]:
for i in c:
    masterlist[i]=kalmanfunction(masterlist[i])

In [10]:
#unzip

for i in c:
    df[i+'_x'] , df[i+'_y'] = zip(*masterlist[i])



In [11]:
finaldf = df[dfkalman_col]
finaldf.to_csv('final.csv',index=False)

In [131]:
#concat

finaldf = pd.concat([df[dfkalman_col],dforiginal[coltodrop]],axis=1)

In [132]:
finaldf = finaldf[dforiginal.columns]

In [133]:
finaldf.to_csv('output.csv',index=False)